# Machine Learning to predict number of customers

In [1]:
import pandas as pd
import numpy as np

#Import Training Data
df = pd.read_csv("training_data_customers.csv")
final_df = df.drop(columns="Unnamed: 1")

## Random Forest Model

In [2]:
from sklearn.ensemble import RandomForestRegressor

## Random Forest
# Best parameters found
best_params = {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}

# Create and train the Random Forest model with the best parameters
rf_model = RandomForestRegressor(random_state=42, **best_params)  # **best_params unpacks the dictionary

In [3]:
#Method to add actual customer numbers into the training data
def update_customer_data(new_customer_counts):
    """Updates customer data for shifts 1, 2, and 3 with rounded values.

    Args:
        new_customer_counts: A NumPy array or a list/tuple that can be converted
                             to a NumPy array of customer counts.

    Returns:
        The updated DataFrame.
    """

    global final_df

    new_customer_counts = np.array(new_customer_counts) # Ensure it's a NumPy array
    if len(new_customer_counts) != 3:
        raise ValueError("new_customer_counts must be a list or tuple of length 3.")


    rounded_counts = np.round(new_customer_counts).astype(int) # Round and convert to integers

    last_group = final_df['Group'].max()
    if pd.isna(last_group):
        new_group = 1
    else:
        new_group = last_group + 1

    new_rows = []
    for i, customers in enumerate(rounded_counts):
        new_rows.append({'Group': new_group, 'Shift': i + 1, 'Customers': customers})

    final_df = pd.concat([final_df, pd.DataFrame(new_rows)], ignore_index=True)
    return final_df

In [4]:
#Retraining the model
def retraining(final_df):
    """Retrains the model with the updated DataFrame.

    Args:
        final_df: The updated pandas DataFrame.

    Returns:
        The retrained model
    """
    X = final_df[['Shift']]
    y = final_df['Customers']

    rf_model.fit(X, y)  # Retrain the model

    return rf_model

# USE LP to Solve Optimization

In [5]:
import pyoptinterface as poi
from pyoptinterface import highs

## Handle No enough Staff Problem in higher demand
When the demand is high and there aren't enough waiters to meet all the constraints:

1. Relax Constraints <br>
Adjust Shift Requirements: Part-time waiters is allowed to work more than one shift per day.

2. Prioritize Constraints <br>
Relax Less Critical Constraints: Hate Group and special request for specific Shifts will be relaxed to find a feasible solution.
Prioritize Critical Shifts: Ensure that the most critical shifts are fully staffed

2. Adding more waiters or more relaxed constraints - Not Implemented <br>
Relax Less Critical Constraints: Hate Group and special request for specific Shifts will be relaxed to find a feasible solution.
Prioritize Critical Shifts: Ensure that the most critical shifts are fully staffed

In [26]:
#Waiter Type
waiter_types = ["Fulltime", "Parttime"]
waite_total = {"Fulltime": 5, "Parttime": 5}

# Waiter Name
waiter_name = {
    "Fulltime": ["Ana", "Bob", "Alice", "Putri", "Lala"],
    "Parttime": ["Laura", "Bill", "Feni", "Stefi", "Johanes"]
}

shifts = [1, 2, 3]
capacity_waiter = 20

# Initialize the model and decision variables
model = highs.Model()

# Define variables for each waiter in each shift
waiter_vars = {}
for waiter_type in waiter_types:
    for waiter in waiter_name[waiter_type]:
        for shift in shifts:
            var_name = f"{waiter}_{shift}"
            waiter_vars[var_name] = model.add_variable(lb=0, ub=1, domain=poi.VariableDomain.Integer, name=var_name)

# List of waiters that can work in each shift
waiters_per_shift = {
    1: ["Ana", "Bob", "Putri", "Lala", "Laura", "Bill",  "Johanes", "Stefi"],
    2: ["Ana", "Bob", "Alice", "Putri", "Lala", "Johanes", "Stefi", "Feni" ],
    3: ["Ana", "Bob", "Alice", "Putri", "Lala",  "Stefi", "Feni"]
}

# Define Group of people who prefer not working with people in another group 
group_A = ["Ana", "Bob", "Alice"]
group_B = ["Putri", "Lala", "Laura"]

def solve_scheduling_problem(waiter_vars, waiter_availability, customer_demands, relax_constraints=False):
    """
    Solves the scheduling problem for assigning waiters to shifts based on various constraints.

    Parameters:
    waiter_vars (dict): Dictionary to store the decision variables for each waiter in each shift.
    waiter_availability (dict): Dictionary indicating the availability of each waiter.
    relax_constraints (bool): Flag to indicate whether to relax certain constraints for feasibility.
    customer_demands (dict): Dictionary indicating the number of customer demands

    Returns:
    model: The optimized model with the scheduling solution.

    Description:
    This function creates and solves an optimization model to assign waiters to shifts while satisfying
    several constraints. The constraints include:
    1. Each full-time waiter can work at most 2 shifts per day (or 3 if constraints are relaxed).
    2. Each part-time waiter can work at most 1 shift per day (or 2 if constraints are relaxed).
    3. The total capacity in each shift must meet or exceed customer demands.
    4. Each shift must have at least 2 waiters.
    5. Only specific waiters can work in each shift.
    6. People in Group A and Group B cannot work together in the same shift.
    7. Only available waiters can be assigned to shifts.

    The objective of the model is to minimize the total number of waiters assigned to shifts.

    The function returns the optimized model with the scheduling solution.
    """
    model = highs.Model()  # Create a new instance of the model

    # Define variables for each waiter in each shift
    for waiter_type in waiter_types:
        for waiter in waiter_name[waiter_type]:
            for shift in shifts:
                var_name = f"{waiter}_{shift}"
                waiter_vars[var_name] = model.add_variable(lb=0, ub=1, domain=poi.VariableDomain.Integer, name=var_name)
    
    # Prioritize full-time waiters
    fulltime_waiters = waiter_name["Fulltime"]
    parttime_waiters = waiter_name["Parttime"]
    
    # Constraint 1: Each full-time waiter can work at most 2 shifts per day
    for waiter in fulltime_waiters:
        model.add_linear_constraint(
            poi.quicksum(waiter_vars[f"{waiter}_{shift}"] for shift in shifts),
            poi.Leq,
            2,
            name=f"{waiter}_fulltime_max_two_shifts"
        )
    
    # Constraint 2: Each part-time waiter can work at most 1 shift per day
    for waiter in parttime_waiters:
        model.add_linear_constraint(
            poi.quicksum(waiter_vars[f"{waiter}_{shift}"] for shift in shifts),
            poi.Leq,
            1 if not relax_constraints else 2,  # Relax constraint if needed
            name=f"{waiter}_parttime_max_one_shift"
        )
    
    # Constraint 3: The total capacity in each shift must meet or exceed customer demands
    for shift in shifts:
        model.add_linear_constraint(
            poi.quicksum(waiter_vars[f"{waiter}_{shift}"] * capacity_waiter for waiter in fulltime_waiters + parttime_waiters),
            poi.Geq,
            customer_demands[shift],
            name=f"shift_{shift}_demand"
        )
    
    # Constraint 4: Each shift must have at least 2 waiters
    for shift in shifts:
        model.add_linear_constraint(
            poi.quicksum(waiter_vars[f"{waiter}_{shift}"] for waiter in fulltime_waiters + parttime_waiters),
            poi.Geq,
            2,
            name=f"shift_{shift}_min_two_waiters"
        )
    
    # Constraint 5: Only specific waiters can work in each shift
    if not relax_constraints:  # Relax Constraint 2, less critical prioritization
        for shift in shifts:
            for waiter in fulltime_waiters + parttime_waiters:
                if waiter not in waiters_per_shift[shift]:
                    model.add_linear_constraint(
                        waiter_vars[f"{waiter}_{shift}"],
                        poi.Eq,
                        0,
                        name=f"{waiter}_not_in_shift_{shift}"
                    )
    
    # Constraint 6: People in Group A and Group B cannot work together in the same shift
    if not relax_constraints:  # Relax Constraint 2, less critical prioritization
        for shift in shifts:
            for waiter_A in group_A:
                for waiter_B in group_B:
                    model.add_linear_constraint(
                        waiter_vars[f"{waiter_A}_{shift}"] + waiter_vars[f"{waiter_B}_{shift}"],
                        poi.Leq,
                        1,
                        name=f"group_A_B_not_together_shift_{shift}"
                    )
    
    # Constraint 7: Only available waiters can be assigned to shifts
    for waiter in fulltime_waiters + parttime_waiters:
        if not waiter_availability[waiter]:
            for shift in shifts:
                model.add_linear_constraint(
                    waiter_vars[f"{waiter}_{shift}"],
                    poi.Eq,
                    0,
                    name=f"{waiter}_not_available"
                )

    # Objective: Minimize the total number of waiters assigned
    model.set_objective(
        poi.quicksum(waiter_vars[var] for var in waiter_vars),
        poi.ObjectiveSense.Minimize
    )

    model.set_model_attribute(poi.ModelAttribute.Silent, False)
    model.optimize()

    return model

# MESA Restaurant Model

In [33]:
import math
import random
from mesa import Agent, Model
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names, but RandomForestRegressor was fitted with feature names")

class Waiter(Agent):
    def __init__(self, model, is_fulltime, name):
        super().__init__(model)  
        self.is_fulltime = is_fulltime
        self.name = name
        self.available = True
        self.consecutive_days_worked = 0  # Track consecutive days worked

    def reset_availability(self):
        if not self.available:
                self.consecutive_days_worked = 0  # Reset consecutive days worked if not available the previous day
        if self.is_fulltime:
            if self.consecutive_days_worked >= 4:
                self.available = False  # Give a day off if worked 4 consecutive days
                self.consecutive_days_worked = 0  # Reset consecutive days worked
            else:
                self.available = True  # Reset availability to True
        else:
            if self.consecutive_days_worked >= 2:
                self.available = False  # Give a day off if worked 2 consecutive days
                self.consecutive_days_worked = 0  # Reset consecutive days worked
            else:
                self.available = True  # Reset availability to True



In [34]:
class RestaurantModel(Model):
    def __init__(self, seed=None):
        super().__init__(seed=seed)
        
        self.demands = {1: 63, 2: 90, 3: 45}  # Initial demand per shift for formatting rules
        self.waiter_types = waiter_types
        self.waite_total = waite_total
        self.waiter_name = waiter_name
        
        # Create waiters with unique IDs
        for name in self.waiter_name["Fulltime"]:
            waiter = Waiter(model=self, is_fulltime=True, name=name) #Agent Waiter Initialization
            self.agents.add(waiter)
        for name in self.waiter_name["Parttime"]:
            waiter = Waiter(model=self, is_fulltime=False, name=name)
            self.agents.add(waiter)

        # Initialize the retrained model
        self.retrained_model = retraining(final_df)

        # Initialize step count
        self.step_count = 0
    
    def step(self):
        self.step_count += 1
        print(f"\n--- Day {self.step_count} ---")   
            
        # Reset availability for all waiters
        for waiter in self.agents:
            waiter.reset_availability()

        # Print available waiters for the day
        available_waiters = [waiter.name for waiter in self.agents if waiter.available]
        print(f"Available waiters for the day: {', '.join(available_waiters)}")

        # Recalculate predicted demands
        customer_data = np.array([[1], [2], [3]])
        new_predictions = self.retrained_model.predict(customer_data)
        self.demands = {key: new_predictions[i] for i, key in enumerate(self.demands.keys())} #Use Predicted Demand (On, Off)

         # Print the number of predicted customers for each shift
        for shift, demand in self.demands.items():
            print(f"Predicted customers for shift {shift}: {round(demand, 3)}")
        
        # Prepare data for the optimization problem
        waiter_vars = {}
        customer_demands = self.demands

        # Collect availability information
        waiter_availability = {waiter.name: waiter.available for waiter in self.agents}

        model= highs.Model()  # Create a new instance of the model

        # Define variables for each waiter in each shift
        for waiter_type in waiter_types:
            for waiter in waiter_name[waiter_type]:
                for shift in shifts:
                    var_name = f"{waiter}_{shift}"
                    waiter_vars[var_name] = model.add_variable(lb=0, ub=1, domain=poi.VariableDomain.Integer, name=var_name)

        # Solve the scheduling problem
        model = solve_scheduling_problem(waiter_vars, waiter_availability, customer_demands, relax_constraints=False)
        termination_status = model.get_model_attribute(poi.ModelAttribute.TerminationStatus)

        if termination_status != poi.TerminationStatusCode.OPTIMAL:
            print("No optimal solution found, trying with relaxed constraints...")
            model = solve_scheduling_problem(waiter_vars, waiter_availability, customer_demands, relax_constraints=True)
            termination_status = model.get_model_attribute(poi.ModelAttribute.TerminationStatus)
            
        # Initialize schedule variable
        schedule = {shift: [] for shift in shifts}

        # Print the schedule if a solution is found
        if termination_status == poi.TerminationStatusCode.OPTIMAL:
            schedule = {shift: [] for shift in shifts}
            for var in waiter_vars:
                if model.get_value(waiter_vars[var]) > 0.5:  # If the waiter is assigned to the shift
                    waiter, shift = var.rsplit('_', 1)
                    schedule[int(shift)].append(waiter)

            for shift in shifts:
                print(f"Shift {shift}: {', '.join(schedule[shift])}, {len(schedule[shift])} Waiters")
        else:
            print("No feasible solution found, even with relaxed constraints.")

        # Update training data with new predictions and retrain the model
        actual_data = new_predictions  # For now using the prediction result, later the actual customer number muss be assigned here. e.g. [30,70,50]
        final_df = update_customer_data(actual_data)
        self.retrained_model = retraining(final_df)

        # Update consecutive days worked for each waiter
        if schedule:
            assigned_waiters = {waiter for shift in schedule.values() for waiter in shift}
            for waiter in self.agents:
                if waiter.name in assigned_waiters:
                    waiter.consecutive_days_worked += 1
                else:
                    waiter.consecutive_days_worked = 0


# Run the model for multiple days
model1 = RestaurantModel(70)  # Seed ensures reproducibility
for _ in range(7):  # Simulate 5 days
    model1.step()


--- Day 1 ---
Available waiters for the day: Ana, Bob, Alice, Putri, Lala, Laura, Bill, Feni, Stefi, Johanes
Predicted customers for shift 1: 47.695
Predicted customers for shift 2: 60.083
Predicted customers for shift 3: 30.299
Shift 1: Ana, Bob, Bill, 3 Waiters
Shift 2: Putri, Lala, Stefi, Johanes, 4 Waiters
Shift 3: Lala, Feni, 2 Waiters

--- Day 2 ---
Available waiters for the day: Ana, Bob, Alice, Putri, Lala, Laura, Bill, Feni, Stefi, Johanes
Predicted customers for shift 1: 47.695
Predicted customers for shift 2: 60.082
Predicted customers for shift 3: 30.298
Shift 1: Ana, Bob, Bill, 3 Waiters
Shift 2: Putri, Lala, Stefi, Johanes, 4 Waiters
Shift 3: Lala, Feni, 2 Waiters

--- Day 3 ---
Available waiters for the day: Ana, Bob, Alice, Putri, Lala, Laura
Predicted customers for shift 1: 47.696
Predicted customers for shift 2: 60.082
Predicted customers for shift 3: 30.299
No optimal solution found, trying with relaxed constraints...
Shift 1: Alice, Lala, Laura, 3 Waiters
Shift 2: 